In [2]:
import torch
from torch import optim
from torch.utils.data import DataLoader
from torchmetrics.functional.text import bleu_score
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import lightning.pytorch as pl
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import os
os.environ['http_proxy'] = 'http://proxy.stc:3128'
os.environ['https_proxy'] = 'http://proxy.stc:3128'
os.environ['ftp_proxy'] = 'http://proxy.stc:3128'

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [10]:
# data = pd.read_csv('dialogues.tsv', sep='\t')
# data.head()

,persona_1_profile,persona_2_profile,dialogue
0,<span class=participant_1>У меня любимая работ...,<span class=participant_2>Ищу принца.<br />Вед...,<span class=participant_2>Пользователь 2: Прив...
1,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />У м...,<span class=participant_1>Пользователь 1: Прив...
2,<span class=participant_1>Я купила дом<br />Я ...,<span class=participant_2>Я пою в караоке<br /...,<span class=participant_1>Пользователь 1: Прив...
3,<span class=participant_1>я врач и женат<br />...,<span class=participant_2>Я мальчик<br />Я учу...,<span class=participant_2>Пользователь 2: Здра...
4,<span class=participant_1>Я школьница.<br />Я ...,<span class=participant_2>Я простоват.<br />Лю...,<span class=participant_1>Пользователь 1: Прив...


In [11]:
# def dialogue_clearing(string):
#     '''
#     Clear text from special characters
#     Args:
#         string (string): one dialog text
    
#     Return
#         list of senteces
#     '''
#     clear_dialogue = []
#     for s in string.split('</span>'):
#         s = re.sub('<span class=participant_1>', '', s)
#         s = re.sub('<span class=participant_2>', '', s)
#         s = re.sub(r'^<br />', '', s)
#         s = re.sub('<br />', ' ', s)
#         s = re.sub(' ,', ',', s)
#         s = re.sub(' \.', '.', s)
#         s = re.sub(',', ', ', s)
#         s = re.sub('  ', ' ', s)
#         s = re.sub(r'\r\n', ' ', s)
#         s = re.sub('"', '', s)
#         s = re.sub('Пользователь ', '', s)
#         s = re.sub('Пользователь ', '', s)
        
#         if len(s) != 0:
#             clear_dialogue.append(s)
    
#     return clear_dialogue

In [4]:
# test = data.dialogue.iloc[0]
# for s in dialogue_clearing(test):
#     print(s)

2: Привет) расскажи о себе
1: Привет) под вкусный кофеек настроение поболтать появилось )
2: Что читаешь? Мне нравится классика
2: Я тоже люблю пообщаться
1: Люблю животных, просто обожаю, как и свою работу)
1: Я фантастику люблю
2: А я выращиваю фиалки
2: И веду здоровый и активный образ жизни!
1: Ух ты, интересно.
2: Ты случайно не принц на белом коне? Я его очень жду..
1: А у меня из хобби каждую неделю тусить с моим лучшим другом)


In [15]:
# def dialogue_concat(sentences_list):
#     '''
#     Concatenate several consecutive sentences of one user to one sequence
    
#     Args:
#         sentences_list (list): list of sentences for one dialogue
        
#     Return:
#         list of sentences
#     '''

#     clear_dialogue_concat = [sentences_list[0]]
#     for i in range(1, len(sentences_list)):
#         s = sentences_list[i]
#         if clear_dialogue_concat[-1][:2] == s[:2]:
#             clear_dialogue_concat[-1] = clear_dialogue_concat[-1] + ' ' + s[3:] 
#         else:
#             clear_dialogue_concat.append(s)
            
#     return clear_dialogue_concat

In [6]:
# dialogue = dialogue_clearing(test)
# for s in dialogue_concat(dialogue):
#     print(s)

2: Привет) расскажи о себе
1: Привет) под вкусный кофеек настроение поболтать появилось )
2: Что читаешь? Мне нравится классика Я тоже люблю пообщаться
1: Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю
2: А я выращиваю фиалки И веду здоровый и активный образ жизни!
1: Ух ты, интересно.
2: Ты случайно не принц на белом коне? Я его очень жду..
1: А у меня из хобби каждую неделю тусить с моим лучшим другом)


In [16]:
# def swap_users(sentence_list):
#     '''
#     Swap user1 and user2
    
#     Args:
#         sentences_list (list): list of sentences for one dialogue
        
#     Return:
#         list of senteces
#     '''

#     dialogue = []
#     for s in sentence_list:
#         if s.find('2:') == 0:
#             s = re.sub('2:', '1:', s)
#             dialogue.append(s)
#             continue
    
#         if s.find('1:') == 0:
#             s = re.sub('1:', '2:', s)
#             dialogue.append(s)
#             continue
    
#     return dialogue

In [50]:
# import random


# test = data.dialogue.iloc[random.randint(0, len(data))]
# dialogue = dialogue_clearing(test)
# dialogue = dialogue_concat(dialogue)
# for s in swap_users(dialogue):
#     print(s)

1: Привеет
2: Привет, как тебя зовут?
1: Меня Настя зовут, а тебя?
2: А я Катя) у меня лучшая подруга Настя, мы вместе плаванием занимаемся)
1: О, класс. А у меня нет лучших подруг. Я вообще не успеваю с кем то дружить. 11 класс- это нечто.
2: Я тебя понимаю, сама в 10м учусь Приходится ещё английским дополнительно заниматься для поступления
1: Куда собираешься поступать?
2: На Гостиничное дело, там требуют знание языка ещё Все времени нет свободно даже в любимый майнкрафт поиграть ))) Чем увлекаешься?
1: Круто, а я в медицинский университет, хочу врачом стать. Просто мечтаю об этом. Это же так здорово людям помогать, жизни их спасать, да и вообще полезная это профессия Какие твои любимые уроки в школе?
2: Да, это круто))) Биология) наверное потому что мама у меня ее ведёт ))) хотим, кстати на море с ней летом, никогда не была... она говорит оно такое же синее как мои глаза 😄


In [9]:
# untokenized_data = []
# dialogue_len = []
# tmp = []
# for i in tqdm(range(data.shape[0])):
#     dialogue = data.dialogue.iloc[i]
    
#     dialogue = dialogue_clearing(dialogue)
#     dialogue = dialogue_concat(dialogue)
    
#     if dialogue[0].find('2:') == 0:
#         dialogue = swap_users(dialogue)
    
#     s = re.sub('2:', '1:', s)
#     for i, repl in enumerate(dialogue):
#         if repl[:2] == '1:':
#             dialogue[i] = '@@ПЕРВЫЙ@@' + repl[2:]
#         elif repl[:2] == '2:':
#             dialogue[i] = '@@ВТОРОЙ@@' + repl[2:]    
#     tmp.append(dialogue)

    
    
#     # # augmentation
#     # if (len(dialogue[2:]) >= 8): tmp.append(dialogue[2:])
#     # if (len(dialogue[4:]) >= 8): tmp.append(dialogue[4:])
#     # if (len(dialogue[6:]) >= 8): tmp.append(dialogue[6:])
#     # if (len(dialogue[8:]) >= 8): tmp.append(dialogue[8:])
#     # if (len(dialogue[10:]) >= 8): tmp.append(dialogue[10:])
#     # if (len(dialogue[12:]) >= 8): tmp.append(dialogue[12:])
#     # if (len(dialogue[14:]) >= 8): tmp.append(dialogue[14:])
#     # if (len(dialogue[16:]) >= 8): tmp.append(dialogue[16:])    
        
#     for d in tmp:    
#         string = ' '.join(d)
#         string = string.strip()
#         string = string + '\n'
#         dialogue_len.append(len(d))
#         untokenized_data.append(string)
        
#     tmp.clear()
    
#     # random.Random(13).shuffle(untokenized_data)
    
# with open('dialogues.txt', 'w+', encoding='utf-8') as file:
#      file.writelines(untokenized_data)

100%|██████████| 10013/10013 [00:02<00:00, 4385.73it/s]


In [10]:
# soup = BeautifulSoup(pd.read_table('dialogues.tsv')["dialogue"].iloc[0])

In [11]:
# import pandas as pd

# context = ''

# for el in BeautifulSoup(pd.read_table('dialogues.tsv')["dialogue"].iloc[0]).findChildren():
#     if el.name == 'span':
#         context += el.text + '\n'

#     # print(el.get(''))
# print(context)

In [3]:
class GenModel(pl.LightningModule):
    def __init__(self, model_name_or_path):
        super().__init__()
        self.save_hyperparameters()
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, truncation_side="left", padding_side='left')
        self.model = AutoModelForCausalLM.from_pretrained(model_name_or_path, pad_token_id=self.tokenizer.eos_token_id)
        self.tokenizer.pad_token = self.tokenizer.eos_token
    
    def training_step(self, batch, batch_idx):
        loss = self.model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label_input_ids"]).loss
        self.log("train_loss", loss.item(), prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        val_loss = self.model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label_input_ids"]).loss
        self.log("val_loss", val_loss.item(), prog_bar=True)
        return val_loss
    
    def predict_step(self, batch, batch_idx):
        outputs = self.model.generate(batch["input_ids"], attention_mask=batch["attention_mask"])
        
        preds = []
        for i in range(len(outputs)):
            pred = self.tokenizer.decode(outputs[i], skip_special_tokens=True)
            preds.append(pred)
        
        return preds
    
    def test_step(self, batch, batch_idx):
        
        outputs = self.model.generate(batch["input_ids"], attention_mask=batch["attention_mask"]) # 'length_penalty' for long messages
        
        preds = []
        for i in range(len(outputs)):
            pred = self.tokenizer.decode(outputs[i], skip_special_tokens=True)
            preds.append(pred)
        
        bleu = bleu_score(preds, batch["summaries"], n_gram=1)
        self.log("bleu", bleu.item(), prog_bar=True)
        return bleu
    
    def infer(self, text):
        inputs = self.tokenizer(text, return_tensors='pt')
        generated_token_ids = self.model.generate(
            **inputs,
            top_k=10,
            top_p=0.95,
            num_beams=3,
            num_return_sequences=1,
            do_sample=True,
            no_repeat_ngram_size=2,
            temperature=1.2,
            repetition_penalty=1.2,
            length_penalty=1.0,
            eos_token_id=50257,
            max_new_tokens=40
        )

        context_with_response = [self.tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
        return context_with_response
        
    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=2e-5)
        return optimizer

In [3]:
def tokenize(tokenizer, row):
    row = tokenizer(row, max_length=100, padding='max_length', truncation=True, return_tensors="pt")
    return row

with open('dialogues.txt', 'r', encoding='utf-8') as file:
    untokenized_data = file.readlines() 

untokenized_data = [s.strip() for s in untokenized_data]

dialogues = []

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-small')

import numpy as np

lens = []

for i, data in enumerate(untokenized_data):
    pairs = []
    for first, second in zip(re.findall(r'@@ПЕРВЫЙ@@[^@]*', data), re.findall(r'@@ВТОРОЙ@@[^@]*', data)):
        pairs.append( (tokenize(tokenizer, first.strip()), tokenize(tokenizer, second.strip())) )

    lens.append(len(pairs))
    dialogues.append(pairs)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
print(f'{np.array(lens).min()=}')
print(f'{np.array(lens).mean()=}')
print(f'{np.array(lens).max()=}')

np.array(lens).min()=1
np.array(lens).mean()=8.163287725956257
np.array(lens).max()=48


In [5]:
np.histogram(np.array(lens), bins=50)

(array([   1,   38,  328,  923, 1314, 1451, 1262, 1103,  860,  653,  484,
         393,  298,  227,  160,  130,    0,   81,   62,   59,   33,   27,
          16,   22,   17,   15,   11,    4,    4,   10,    3,    4,    4,
           0,    6,    1,    2,    1,    2,    0,    0,    1,    0,    0,
           0,    1,    0,    1,    0,    1]),
 array([ 1.  ,  1.94,  2.88,  3.82,  4.76,  5.7 ,  6.64,  7.58,  8.52,
         9.46, 10.4 , 11.34, 12.28, 13.22, 14.16, 15.1 , 16.04, 16.98,
        17.92, 18.86, 19.8 , 20.74, 21.68, 22.62, 23.56, 24.5 , 25.44,
        26.38, 27.32, 28.26, 29.2 , 30.14, 31.08, 32.02, 32.96, 33.9 ,
        34.84, 35.78, 36.72, 37.66, 38.6 , 39.54, 40.48, 41.42, 42.36,
        43.3 , 44.24, 45.18, 46.12, 47.06, 48.  ]))

In [4]:
class GenDataModule(pl.LightningDataModule):
    def __init__(self, model_name_or_path, filename):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        # self.tokenizer.pad_token = self.tokenizer.eos_token
        self.filename = filename
        
    def prepare_data(self):
        with open(self.filename, 'r', encoding='utf-8') as file:
            untokenized_data = file.readlines() 

        untokenized_data = [s.strip() for s in untokenized_data]

        dialogues = []

        for i, data in enumerate(untokenized_data):
            pairs = []
            for first, second in zip(re.findall(r'@@ПЕРВЫЙ@@[^@]*', data), re.findall(r'@@ВТОРОЙ@@[^@]*', data)):
                pairs.append( (self.tokenize(first.strip()), self.tokenize(second.strip())) )
                
                if len(pairs) == 8: break
            dialogues.append(pairs)

        train, test = train_test_split(dialogues, test_size=0.1)

        self.ds = {
            'train': train,
            'validation': test,
            'test': test
        }

        
    def train_dataloader(self):
        train_split = self.ds["train"]
        return DataLoader(train_split, batch_size=1, collate_fn=self.collate, num_workers=os.cpu_count() - 1)
    
    def val_dataloader(self):
        val_split = self.ds["validation"]
        return DataLoader(val_split, batch_size=1, collate_fn=self.collate, num_workers=os.cpu_count() - 1)
    
    def predict_dataloader(self):
        val_split = self.ds["validation"]
        return DataLoader(val_split, batch_size=1, collate_fn=self.collate, num_workers=os.cpu_count() - 1)
    
    def test_dataloader(self):
        test_split = self.ds["test"]
        return DataLoader(test_split, batch_size=1, collate_fn=self.collate, num_workers=os.cpu_count() - 1)
    
    def tokenize(self, row):
        row = self.tokenizer(row, max_length=128, padding='max_length', truncation=True, return_tensors="pt")
        return row
    
    def collate(self, dialogues):
        input_ids = torch.stack([torch.tensor(first["input_ids"].squeeze().detach()) for dialogue in dialogues for first, second in dialogue])
        attention_mask = torch.stack([torch.tensor(first["attention_mask"].squeeze()) for dialogue in dialogues for first, second in dialogue])
        label_input_ids = torch.stack([torch.tensor(second["input_ids"].squeeze()) for dialogue in dialogues for first, second in dialogue])
        
        return {"input_ids": input_ids, "attention_mask": attention_mask, "label_input_ids": label_input_ids}

In [4]:
gen_model = GenModel("tinkoff-ai/ruDialoGPT-small")
gen_dm = GenDataModule("tinkoff-ai/ruDialoGPT-small", filename='dialogues.txt')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
trainer = pl.Trainer(max_time="00:11:00:00", default_root_dir='chatgpt_lightning_logs')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [47]:
inp = """@@ПЕРВЫЙ@@ Здравствуйте!
@@ВТОРОЙ@@ Привет
@@ПЕРВЫЙ@@ Как Вас зовут?
@@ВТОРОЙ@@ Алексей, а Вас?
@@ПЕРВЫЙ@@ Виктория. Сколько Вам лет?
@@ВТОРОЙ@@ 
"""

def chat(text, model):
    answer = model.infer(text.replace('\n', ' '))[0].split('@@ВТОРОЙ@@')[-1]
    return answer

chat(inp, gen_model).strip()

'   Я не знаю, сколько мне лет, но я не помню, чтобы меня так звали.  Но я бы хотела быть Викторией 😉🙈😊💕🏻'

In [52]:
inp = """@@ПЕРВЫЙ@@ Привеет
@@ВТОРОЙ@@ Привет, как тебя зовут?
@@ПЕРВЫЙ@@ Меня Настя зовут, а тебя?
@@ВТОРОЙ@@ А я Катя) у меня лучшая подруга Настя, мы вместе плаванием занимаемся)
@@ПЕРВЫЙ@@ О, класс. А у меня нет лучших подруг. Я вообще не успеваю с кем то дружить. 11 класс- это нечто.
@@ВТОРОЙ@@ 
"""
chat(inp, gen_model).strip()

'Я тоже не особо дружу с девочками, потому что я не могу с ними общаться. Это не друзья, это просто люди, которые не умеют общаться :(  Удачи тебе!  '

In [6]:
trainer.fit(model=gen_model, datamodule=gen_dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.917   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/tmp/ipykernel_3144513/3548661934.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.stack([torch.tensor(first["input_ids"].squeeze().detach()) for dialogue in dialogues for first, second in dialogue])
/tmp/ipykernel_3144513/3548661934.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.stack([torch.tensor(first["attention_mask"].squeeze()) for dialogue in dialogues for first, second in dialogue])
/tmp/ipykernel_3144513/3548661934.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_input_

Training: 0it [00:00, ?it/s]

/tmp/ipykernel_3144513/3548661934.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.stack([torch.tensor(first["input_ids"].squeeze().detach()) for dialogue in dialogues for first, second in dialogue])
/tmp/ipykernel_3144513/3548661934.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.stack([torch.tensor(first["attention_mask"].squeeze()) for dialogue in dialogues for first, second in dialogue])
/tmp/ipykernel_3144513/3548661934.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = 

...

Validation: 0it [00:00, ?it/s]

/tmp/ipykernel_3144513/3548661934.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.stack([torch.tensor(first["input_ids"].squeeze().detach()) for dialogue in dialogues for first, second in dialogue])
/tmp/ipykernel_3144513/3548661934.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.stack([torch.tensor(first["attention_mask"].squeeze()) for dialogue in dialogues for first, second in dialogue])
/tmp/ipykernel_3144513/3548661934.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = 

Validation: 0it [00:00, ?it/s]

/tmp/ipykernel_3144513/3548661934.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.stack([torch.tensor(first["input_ids"].squeeze().detach()) for dialogue in dialogues for first, second in dialogue])
/tmp/ipykernel_3144513/3548661934.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.stack([torch.tensor(first["input_ids"].squeeze().detach()) for dialogue in dialogues for first, second in dialogue])
/tmp/ipykernel_3144513/3548661934.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mas

Validation: 0it [00:00, ?it/s]

In [6]:
inp = """@@ПЕРВЫЙ@@ Здравствуйте!
@@ВТОРОЙ@@ Привет
@@ПЕРВЫЙ@@ Как Вас зовут?
@@ВТОРОЙ@@ Алексей, а Вас?
@@ПЕРВЫЙ@@ Виктория. Сколько Вам лет?
@@ВТОРОЙ@@ 
"""

def chat(text, model):
    answer = model.infer(text.replace('\n', ' '))[0].split('@@ВТОРОЙ@@')[-1]
    return answer

chat(inp, gen_model).strip()

'Мне 24. Я не знаю, сколько вам лет, но я не вижу в Вас ничего, что могло бы вас заинтересовать.  Удачи! ) 💋🏻❤�'

In [7]:
inp = """@@ПЕРВЫЙ@@ Привеет
@@ВТОРОЙ@@ Привет, как тебя зовут?
@@ПЕРВЫЙ@@ Меня Настя зовут, а тебя?
@@ВТОРОЙ@@ А я Катя) у меня лучшая подруга Настя, мы вместе плаванием занимаемся)
@@ПЕРВЫЙ@@ О, класс. А у меня нет лучших подруг. Я вообще не успеваю с кем то дружить. 11 класс- это нечто.
@@ВТОРОЙ@@ 
"""
chat(inp, gen_model).strip()

'А что ты делаешь в 11 классе? И почему у тебя нет подруг? :D  У меня есть подруга, но я с ней не дружу :( ^^  Но это не важно'

In [9]:
%load_ext tensorboard
%tensorboard --logdir chatgpt_lightning_logs/lightning_logs/